# Projet maison n° 4

In [219]:
# imports
import pandas as pd

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python qui produit un unique DataFrame avec tous les fichiers en utilisant pandas, pas de bash :)

Ordre et noms des colonnes : 'year', 'name', 'gender', 'births'

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini par les différents fichiers (voir la documentation du dataset).

In [440]:
def sort_txt(file=str()):
    columns=['name','gender','births']
    df=pd.read_csv(file, sep=',',names=columns)
    df['year']=file
    df['year']=df['year'].str.extract('([0-9].{3})')
    df = df.astype({"year": int, "name": str, "gender": str, "births": int})
    return df[['year', 'name', 'gender', 'births']]

In [441]:
# names us
def df_names_us():
    path='./names'
    files = sorted(os.listdir(path))
    res=sort_txt(path+'/'+files[1])
    for name in files[2:]:
        res=res.append(sort_txt(path+'/'+name),ignore_index=True)
    return res

In [442]:
res=df_names_us()
len(res)

1989401

In [443]:
isinstance(res.index, pd.core.indexes.range.RangeIndex)

True

In [444]:
list(res.columns)== ['year', 'name', 'gender', 'births']

True

In [445]:
res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1989401 entries, 0 to 1989400
Data columns (total 4 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   year    int32 
 1   name    object
 2   gender  object
 3   births  int32 
dtypes: int32(2), object(2)
memory usage: 45.5+ MB


## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2019_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction Python qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : year, name, gender, births
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne 'gender'
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : années (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [446]:
# names fr
def df_names_fr():
    file='nat2019.csv'
    df_f=pd.read_csv(file, sep=';',header=0)
    df_f.columns=['gender','name','year','births']
    mapping = {2: 'F', 1: 'M'}
    df_f['gender'] = df_f['gender'].map(mapping)
    df_f=df_f.loc[~df_f.year.str.startswith("X",na=False)]
    df_f=df_f.astype({"year": int, "name": str, "gender": str, "births": int})
    df_f=df_f.loc[~df_f['name'].str.startswith('_',na=False)]
    df_f['name']=df_f['name'].apply(lambda x:x.capitalize())
    df_f=df_f.loc[df_f['name'].str.len()>=2]
    df_f=df_f[['year', 'name', 'gender', 'births']]
    df_f=df_f.sort_values(['year', 'gender', 'births', 'name'],ascending=[True,True,False,True])
    df_f=df_f.reset_index(drop=True)
    return df_f

In [447]:
res_f=df_names_fr()
res_f.head()

,year,name,gender,births
0,1900,Marie,F,48713
1,1900,Jeanne,F,13981
2,1900,Marguerite,F,8058
3,1900,Germaine,F,6981
4,1900,Louise,F,6696


In [448]:
isinstance(res_f.index, pd.core.indexes.range.RangeIndex)

True

In [449]:
df2.loc[df2.isnull().any(axis=1)].empty

True

## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/browseBox.do?node=5385566

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3.
 
1. Télécharger le dataset des taux de change : http://webstat.banque-france.fr/fr/downloadFile.do?id=5385698&exportType=csv


2. Implémenter une fonction qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction pd.to_datetime() avec le format '%d/%m/%Y'). Les colonnes du DataFrame doivent correspondre aux devises.

In [450]:
del df_t

NameError: name 'df_t' is not defined

In [ ]:
devises=['CHF', 'GBP', 'USD']

In [ ]:
# taux de change
def df_taux_change(devises):
    path='./taux'
    file='Webstat_Export_20201031.csv'
    df_t=pd.read_csv(path+'/'+file, sep=';',header=0,skiprows=[0,1,3,4,5])
    df_t=df_t.rename(columns={'Unité :':'Date'})
    #format = '%d/%m/%Y'
    #df_t['Date']=pd.to_datetime(df_t.Date,format=format)
    df_t['Date']=pd.to_datetime(df_t.Date,infer_datetime_format=True)
    df_t = df_t.set_index(df_t['Date'])
    colonnes=list(df_t.columns)
    cols=[]
    for colonne in colonnes:
        cols.append(colonne.split()[-1].replace('(','').replace(')',''))
    df_t.columns=(cols)
    df_t=df_t[a]
    df_t=df_t.dropna()
    for i in range(len(a)):
        df_t[a[i]]=df_t[a[i]].str.replace(',','.')
        df_t=df_t.loc[~df_t[a[i]].str.contains('-')]
        df_t[a[i]]=pd.to_numeric(df_t[a[i]])
    return df_t

In [ ]:
res_t=df_taux_change(devises)
res_t.head()

In [ ]:
res_t.isnull().any().any()

In [ ]:
list(res_t.columns), ['CHF', 'GBP', 'USD']

In [ ]:
res_t.dtypes 

In [ ]:
isinstance(res_t.index, pd.core.indexes.datetimes.DatetimeIndex)

In [ ]:
res_t.loc[res_t.isnull().any(axis=1)].empty

### Tests

In [451]:
import unittest

class Lesson4Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 1989401)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 615912)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test names
        self.assertTrue(df.loc[df['name'].str.contains('^[A-Z]+(?:-[A-Z]+)?$')].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df['gender']=='F']) + len(df.loc[df['gender']=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(['CHF', 'GBP', 'USD'])
        # colonnes
        self.assertEqual(list(df.columns), ['CHF', 'GBP', 'USD'])
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.datetimes.DatetimeIndex))
        # types taux
        self.assertTrue((df.dtypes == 'float').all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)


In [452]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Lesson4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [453]:
# run tests

run_tests()

test_df_names_fr (__main__.Lesson4Tests) ... ok
test_df_names_us (__main__.Lesson4Tests) ... ok
test_df_taux_change (__main__.Lesson4Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 12.066s

OK
